In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity


In [2]:
df = pd.read_csv("/Users/kaustubhmestri/Projects/Zomato Intelligence/data/processed/zomato_with_clusters.csv")
df.head()


,Unnamed: 0,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,...,rating_text,votes,cost_per_person,rating_weighted,cuisine_count,online_delivery,table_booking,price_category,success,cluster
0,0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,...,Excellent,314,550.0,27.612349,3,0,1,Premium,1,1
1,1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,...,Excellent,591,600.0,28.725780,1,0,1,Premium,1,1
2,2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,...,Very Good,270,2000.0,24.649323,4,0,1,Luxury,1,1
3,3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,...,Excellent,365,750.0,28.922903,2,0,0,Premium,1,1
4,4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,...,Excellent,229,750.0,26.102781,2,0,1,Premium,1,1


In [3]:
rec_df = df[[
    "restaurant_name",
    "city",
    "cuisines",
    "cost_per_person",
    "aggregate_rating",
    "online_delivery"
]].copy()


In [4]:
from sklearn.preprocessing import MultiLabelBinarizer

rec_df["cuisines"] = rec_df["cuisines"].apply(lambda x: x.split(", "))

mlb = MultiLabelBinarizer()
cuisine_encoded = mlb.fit_transform(rec_df["cuisines"])

cuisine_df = pd.DataFrame(
    cuisine_encoded,
    columns=mlb.classes_,
    index=rec_df.index
)


In [5]:
feature_matrix = pd.concat([
    cuisine_df,
    rec_df[["cost_per_person", "aggregate_rating", "online_delivery"]]
], axis=1)


In [6]:
similarity = cosine_similarity(feature_matrix)


In [7]:
def recommend_restaurants(
    restaurant_name,
    top_n=5
):
    idx = rec_df[rec_df["restaurant_name"] == restaurant_name].index[0]
    
    sim_scores = list(enumerate(similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    top_indices = [i[0] for i in sim_scores[1:top_n+1]]
    
    return rec_df.iloc[top_indices][
        ["restaurant_name", "city", "aggregate_rating", "cost_per_person"]
    ]


In [8]:
recommend_restaurants("Domino's Pizza")

,restaurant_name,city,aggregate_rating,cost_per_person
897,Chicago Pizza,Faridabad,2.7,300.0
5820,Chicago Pizza,New Delhi,2.6,300.0
4764,Chicago Pizza,New Delhi,3.2,400.0
3229,Da Pizza Corner,New Delhi,2.9,300.0
7045,Pizza Hut,Ranchi,3.5,350.0
